In [1]:
# Imports: 
!pip install neurokit2
import pandas as pd
import numpy as np
import csv
import sys 
sys.path.append("C:\\Users\\tiago\\OneDrive - Universidade de Aveiro\\3- Data Collection1")
import pickle
import neurokit2 as nk
import os

# Functions to acess files: 
from reading_function import * 
from stationarity_function import * 
from neurokit_function import * 


In [2]:
def open_all (csvfile):
    
    # To save participants information:
    index_start_S1 = []    # List of Session1 start indices of all participants
    index_end_S1 = []      # List of Session1 end indices of all participants
    index_start_S2 = []    # List of Session2 start indices of all participants
    index_end_S2 = []      # List of Session2 end indices of all participants
    index_triggers_S1 = [] # List of Session1 triggers indices of all participants
    index_triggers_S2 = [] # List of Session2 triggers indices of all participants
    
    # To save Data: 
    # id_information: Will be defined later as a dataframe containing informations about the all IDs (Participants) 
    dict_data_S1 = {}
    dict_data_S2 = {}
    dict_emotion_S1 = {}
    dict_emotion_S2 = {}
    b_pre_dict_S1 = {}
    b_pre_dict_S2 = {}
    n_pre_dict_S1= {}
    n_pre_dict_S2 = {}
    h_pre_dict_S1 = {}
    h_pre_dict_S2 = {}
    f_pre_dict_S1 = {}
    f_pre_dict_S2 = {}
    all_s1 = {}
    all_s2 = {}
    baseline_list_S1 = []
    baseline_list_S2 = []
    neutral_list_S1 = []
    neutral_list_S2 = []
    happy_list_S1 = []
    happy_list_S2 = []
    fear_list_S1  = []
    fear_list_S2  = []
    id_code = []
    emotional_sequence = []
    seq_list = []
    session1_filesname = []
    session2_filesname = []
    dates_S1 = []
    dates_S2 = []
    times_S1 = []
    times_S2 = []
    
    with open(csvfile, newline='') as csvfile: # csvfile is a file object so it should be opened with newline=''
        next(csvfile) # The first line have the index which is not what we want here so we jump that first line
        spamreader = csv.reader(csvfile,delimiter=";")
        for row in spamreader: 
            code = row[0]
            session1_name = row[1]
            session2_name = row[2]
            sequence = ['Baseline',row[3],row[4],row[5]]
            seq = row[3][0]+row[4][0]+row[5][0]
            
            id_code.append(code)
            emotional_sequence.append(sequence)
            seq_list.append(seq)
            session1_filesname.append(session1_name)
            session2_filesname.append(session2_name)
            
            # First session: 
            path = r'C:\\Users\\tiago\\OneDrive - Universidade de Aveiro\\3- Data Collection1'
            filepath_S1 = path+'\\'+code+'\\'+session1_name+'.txt'
            # Second session: 
            path = r'C:\\Users\\tiago\\OneDrive - Universidade de Aveiro\\3- Data Collection1'
            filepath_S2 = path+'\\'+code+' S2'+'\\'+session2_name+'.txt'
            
            # Using reading function to read the data file:
            dataS1, headerS1, sampling_rateS1, resolutionS1, dateS1, timeS1, sensorS1, labelS1, columnS1, sleeve_colorS1 = reading_function(filepath_S1)
            dates_S1.append(dateS1) # Adding session1 date
            times_S1.append(timeS1) # Adding session1 time
            dataS2, headerS2, sampling_rateS2, resolutionS2, dateS2, timeS2, sensorS2, labelS2, columnS2, sleeve_colorS2 = reading_function(filepath_S2)
            dates_S2.append(dateS2) # Adding session1 date
            times_S2.append(timeS2) # Adding session1 time
    
            x_nSeq_S1, x_trig_S1, x_EMG_MF_S1, x_EMG_TR_S1, x_EDA_S1, x_ECG_S1=sensor_confirmation(sensorS1,sleeve_colorS1,labelS1,columnS1)
            x_nSeq_S2, x_trig_S2, x_EMG_MF_S2, x_EMG_TR_S2, x_EDA_S2, x_ECG_S2=sensor_confirmation(sensorS2,sleeve_colorS2,labelS2,columnS2)
            
            ######### Sensor confirmation: Session1 #########
            trigger_S1=dataS1[:,x_trig_S1] #Acessing to triggers column
            EMG_MF_S1=dataS1[:,x_EMG_MF_S1] #Acessing to EMG_MF column
            EMG_TR_S1=dataS1[:,x_EMG_TR_S1] #Acessing to EMG_TR column
            EDA_S1=dataS1[:,x_EDA_S1] #Acessing to EDA column
            ECG_S1=dataS1[:,x_ECG_S1] #Acessing to ECG column
            ######### Sensor confirmation: Session2 #########
            trigger_S2=dataS2[:,x_trig_S2] #Acessing to triggers column
            EMG_MF_S2=dataS2[:,x_EMG_MF_S2] #Acessing to EMG_MF column
            EMG_TR_S2=dataS2[:,x_EMG_TR_S2] #Acessing to EMG_TR column
            EDA_S2=dataS2[:,x_EDA_S2] #Acessing to EDA column
            ECG_S2=dataS2[:,x_ECG_S2] #Acessing to ECG column
            
            ######### Find triggers and confirmation: Session1 #########
            index_trig_S1=trigger_function(trigger_S1, sampling_rateS1)
            index_start_S1.append(index_trig_S1[0:4]) # Triggers when set begins
            index_end_S1.append(index_trig_S1[1:5]) # Triggers when set ends
            index_triggers_S1.append(index_trig_S1) # All triggers
            ######### Find triggers and confirmation: Session2 #########
            index_trig_S2=trigger_function(trigger_S2, sampling_rateS2)
            index_start_S2.append(index_trig_S2[0:4]) # Triggers when set begins
            index_end_S2.append(index_trig_S2[1:5]) # Triggers when set ends
            index_triggers_S2.append(index_trig_S2) # All triggers
    
            ######### Rearreange session data #########
            concatenated_dataS1=np.column_stack((EMG_MF_S1,EMG_TR_S1,EDA_S1,ECG_S1)) # Rearrange data (always the same sequence)
            all_s1[code]=concatenated_dataS1
            dict_data_S1[code]=concatenated_dataS1[index_trig_S1[0]:index_trig_S1[4],:]
            ######### Rearreange session data #########
            concatenated_dataS2=np.column_stack((EMG_MF_S2,EMG_TR_S2,EDA_S2,ECG_S2)) # Rearrange data (always the same sequence)
            all_s2[code]=concatenated_dataS2
            dict_data_S2[code]=concatenated_dataS2[index_trig_S2[0]:index_trig_S2[4],:]
   
            ######### List emotional condition #########
            for i in range(0,len(sequence)):
                if sequence[i]=='Baseline':
                    b_start_S1=index_trig_S1[i]
                    b_end_S1=index_trig_S1[i+1]
                    b_pre_dict_S1[code]=concatenated_dataS1[b_start_S1:b_end_S1,:]#retirei index_trig[4]+1 
                    b_start_S2=index_trig_S2[i]
                    b_end_S2=index_trig_S2[i+1]
                    b_pre_dict_S2[code]=concatenated_dataS2[b_start_S2:b_end_S2,:]#retirei index_trig[4]+1 
            
                elif sequence[i]=='Fear':
                    f_start_S1=index_trig_S1[i]
                    f_end_S1=index_trig_S1[i+1]
                    f_pre_dict_S1[code]=concatenated_dataS1[f_start_S1:f_end_S1,:]#retirei index_trig[4]+1
                    f_start_S2=index_trig_S2[i]
                    f_end_S2=index_trig_S2[i+1]
                    f_pre_dict_S2[code]=concatenated_dataS2[f_start_S2:f_end_S2,:]#retirei index_trig[4]+1
            
                elif sequence[i]=='Happy':
                    h_start_S1=index_trig_S1[i]
                    h_end_S1=index_trig_S1[i+1]
                    h_pre_dict_S1[code]=concatenated_dataS1[h_start_S1:h_end_S1,:]#retirei index_trig[4]+1
                    h_start_S2=index_trig_S2[i]
                    h_end_S2=index_trig_S2[i+1]
                    h_pre_dict_S2[code]=concatenated_dataS2[h_start_S2:h_end_S2,:]#retirei index_trig[4]+1
        
                elif sequence[i]=='Neutral':
                    n_start_S1=index_trig_S1[i]
                    n_end_S1=index_trig_S1[i+1]
                    n_pre_dict_S1[code]=concatenated_dataS1[n_start_S1:n_end_S1,:]#retirei index_trig[4]+1
                    n_start_S2=index_trig_S2[i]
                    n_end_S2=index_trig_S2[i+1]
                    n_pre_dict_S2[code]=concatenated_dataS2[n_start_S2:n_end_S2,:]#retirei index_trig[4]+1
                
    baseline_list_S1.append(b_pre_dict_S1)
    fear_list_S1.append(f_pre_dict_S1)
    happy_list_S1.append(h_pre_dict_S1)
    neutral_list_S1.append(n_pre_dict_S1)
    
    dict_emotion_S1['Baseline']=baseline_list_S1
    dict_emotion_S1['Fear']=fear_list_S1
    dict_emotion_S1['Happy']=happy_list_S1
    dict_emotion_S1['Neutral']=neutral_list_S1
                
    id_information_S1=pd.DataFrame({'Participant ID':id_code,'Emotional sequence':emotional_sequence,'Sequence':seq_list,'Triggers index':index_triggers_S1, 'Triggers start':index_start_S1, 'Triggers end':index_end_S1, 'Date S1':dates_S1, 'Time S1':times_S1})
    
    baseline_list_S2.append(b_pre_dict_S2)
    fear_list_S2.append(f_pre_dict_S2)
    happy_list_S2.append(h_pre_dict_S2)
    neutral_list_S2.append(n_pre_dict_S2)
    
    dict_emotion_S2['Baseline']=baseline_list_S2
    dict_emotion_S2['Fear']=fear_list_S2
    dict_emotion_S2['Happy']=happy_list_S2
    dict_emotion_S2['Neutral']=neutral_list_S2
                
    id_information_S2=pd.DataFrame({'Participant ID':id_code,'Emotional sequence':emotional_sequence,'Sequence':seq_list,'Triggers index':index_triggers_S2, 'Triggers start':index_start_S2, 'Triggers end':index_end_S2, 'Date S2':dates_S2, 'Time S2':times_S2})

    return    id_information_S1, dict_emotion_S1, dict_data_S1, id_information_S2, dict_emotion_S2, dict_data_S2 

In [3]:
# Tests for 2 participants:
#id_information_S1, dict_emotion_S1, dict_data_S1, id_information_S2, dict_emotion_S2, dict_data_S2 = open_all('teste.csv')

# Tests to make sure that all is alright:
#print(dataS1.shape)
#print(index_trig_S1)
#print(index_trig_S1[0:4])
#print(index_trig_S1[1:5])
#print(session1_filesname)
#print(concatenated_dataS1[64534:365134,:])   # No ficheiro txt começa sempre num nº diferente de 0, mas a quantidade de valores está coerente e correta
#concatenated_dataS1[b_start_S1:b_end_S1,:]
#print(concatenated_dataS1.shape)  # (2386350, 4)  
#print(dict_emotion_S1['Baseline'])
#print('\n',  session2_filesname)
#print('\n',id_information_S1['Participant ID'])
#print('\n', dict_emotion_S1)
#print('\n', dict_data_S1)
#print(id_information_S2['Participant ID'])
#print('\n', dict_emotion_S2)
#print('\n', dict_data_S2)

# Running for all participants: 
#id_information_S1, dict_emotion_S1, dict_data_S1, id_information_S2, dict_emotion_S2, dict_data_S2 = open_all('data_id.csv') 


In [51]:
# Creating files with Session1 relevant information to keep:
#filename_S1 = 'id_information_S1_file'
#file_S1 = open(filename_S1,'wb')
#pickle.dump(id_information_S1,file_S1)
#file_S1.close()
 
#filename_S1 = 'dict_emotion_S1_file'
#file_S1 = open(filename_S1, 'wb')
#pickle.dump(dict_emotion_S1, file_S1)
#file_S1.close

#filename_S1 = 'dict_data_S1_file'
#file_S1 = open(filename_S1, 'wb')
#pickle.dump(dict_data_S1, file_S1)
#file_S1.close()

In [5]:
# Creating files with Session2 relevant information to keep: 
#filename_S2 = 'id_information_S2_file'
#file_S2 = open(filename_S2,'wb')
#pickle.dump(id_information_S2,file_S2)
#file_S2.close()

#filename_S2 = 'dict_emotion_S2_file'
#file_S2 = open(filename_S2,'wb')
#pickle.dump(dict_emotion_S2,file_S2)
#file_S2.close()

#filename_S2 = 'dict_data_S2_file'
#file_S2 = open(filename_S2,'wb')
#pickle.dump(dict_data_S2,file_S2)
#file_S2.close()

## Participants information data: 

In [3]:
# Sessão 1:
id_information_S1 = pickle.load(open('id_information_S1_file', 'rb'))
dict_data_S1 = pickle.load(open('dict_data_S1_file', 'rb'))
dict_emotion_S1 = pickle.load(open('dict_emotion_S1_file', 'rb'))

# Sessão 2: 
id_information_S2 = pickle.load(open('id_information_S2_file', 'rb'))
dict_data_S2 = pickle.load(open('dict_data_S2_file', 'rb'))
dict_emotion_S2 = pickle.load(open('dict_emotion_S2_file', 'rb'))

print(dict_emotion_S1)

{'Baseline': [{'C001': array([[32764., 32733., 41536., 35817.],
       [32908., 32689., 41555., 35655.],
       [32919., 32553., 41549., 35557.],
       ...,
       [32644., 32741., 44406., 31299.],
       [32744., 32609., 44412., 31215.],
       [32814., 32809., 44423., 31262.]]), 'C002': array([[32896., 32473., 26661., 45949.],
       [32976., 32702., 26660., 47236.],
       [33016., 32676., 26649., 48485.],
       ...,
       [32704., 32654.,  7484., 31188.],
       [32724., 32863.,  7493., 31364.],
       [32692., 32849.,  7502., 31209.]]), 'C003': array([[32688., 33185., 65535., 32237.],
       [32745., 33132., 65535., 32303.],
       [32700., 33004., 65535., 32293.],
       ...,
       [32732., 32292., 25845., 31887.],
       [32772., 32287., 25844., 31784.],
       [32781., 32485., 25847., 31735.]]), 'C004': array([[32722., 32402., 31509., 32128.],
       [32756., 32441., 31517., 32132.],
       [32748., 32263., 31512., 31909.],
       ...,
       [32820., 32829., 36208., 32415.

In [21]:
print(len(dict_emotion_S1['Baseline'][0]['C001'][:,3]))

print(len(dict_emotion_S1['Baseline'][0]['C002'][:,3]))

print(len(dict_emotion_S1['Baseline'][0]['C003'][:,3]))


print(len(dict_emotion_S1['Happy'][0]['C001'][:,3]))

print(len(dict_emotion_S1['Happy'][0]['C002'][:,3]))

print(len(dict_emotion_S1['Happy'][0]['C003'][:,3]))


print(len(dict_emotion_S1['Fear'][0]['C001'][:,3]))

print(len(dict_emotion_S1['Fear'][0]['C002'][:,3]))

print(len(dict_emotion_S1['Fear'][0]['C003'][:,3]))

301229
300601
300631
607898
607126
605936
593385
593000
593558


In [10]:
def all_preprocessed_features (dict_emotion, condition, sampling_rate):
    
    dic_features={}
        
    for participant in dict_emotion[condition][0].keys():  
        
        print(participant)
        
        if (condition=='Happy' and participant=='C028'):
            a=605829
        elif (condition=='Happy' and participant=='C026'):
            a=606905
        elif (condition=='Neutral' and participant=='C014'):
            a=667227
        else:
            a=len(dict_emotion[condition][0][participant][:,0])
            
        # Features
        MF_duration_activity, MF_peak_activity, MF_mean_activity, MF_area_activity, MF_amplitude_activity = signal_process (dict_emotion[condition][0][participant][:,0], 'EMG', sampling_rate, a)
        TR_duration_activity, TR_peak_activity, TR_mean_activity, TR_area_activity, TR_amplitude_activity = signal_process (dict_emotion[condition][0][participant][:,1], 'EMG', sampling_rate, len(dict_emotion[condition][0][participant][:,1]))
        eda_symp, scr_height, scr_amplitude, scr_risetime, scr_recoverytime, eda_tonic, eda_phasic = signal_process (dict_emotion[condition][0][participant][:,2], 'EDA', sampling_rate, len(dict_emotion[condition][0][participant][:,2]))
        ecg_Rpeaks, ecg_rate, t_duration = signal_process (dict_emotion[condition][0][participant][:,3], 'ECG', sampling_rate, len(dict_emotion[condition][0][participant][:,3]))
    
        dic_features[participant]={'MF_duration_activity':MF_duration_activity,'MF_peak_activity':MF_peak_activity,
                                   'MF_mean_activity':MF_mean_activity,'MF_area_activity':MF_area_activity,
                                   'MF_amplitude_activity':MF_amplitude_activity,
                                   'TR_duration_activity':TR_duration_activity,'TR_peak_activity':TR_peak_activity,
                                   'TR_mean_activity':TR_mean_activity,'TR_area_activity':TR_area_activity,
                                   'TR_amplitude_activity':TR_amplitude_activity,
                                   'scr_height':scr_height,'scr_amplitude':scr_amplitude,'scr_risetime':scr_risetime,
                                   'scr_recoverytime':scr_recoverytime, 'eda_tonic':eda_tonic, 'eda_phasic':eda_phasic,
                                   'ecg_rate':ecg_rate,'t_duration':t_duration,
                                   'eda_symp':eda_symp, 'ecg_Rpeaks':ecg_Rpeaks}
      
    
    return dic_features

In [11]:
sampling_rate=1000

In [12]:
dic_features_b_S1 = all_preprocessed_features(dict_emotion_S1, 'Baseline', sampling_rate)

C001
C002
C003
C004
C005
C006
C007
C008
C009
C010
C011
C012
C013
C014
C015
C016
C018
C019
C020
C021
C022
C023
C025
C026
C027
C028
C029
C030
C031


In [13]:
filename = 'dic_baseline_file_S1'
file=open(filename,'wb')
pickle.dump(dic_features_b_S1,file)
file.close()

In [14]:
dic_features_b_S2 = all_preprocessed_features(dict_emotion_S2, 'Baseline', sampling_rate)

C001
C002
C003
C004
C005
C006
C007


C:\Users\tiago\anaconda3\lib\site-packages\numpy\core\fromnumeric.py:3440: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\tiago\anaconda3\lib\site-packages\numpy\core\_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


C008
C009
C010
C011
C012
C013
C014
C015
C016
C018
C019
C020
C021
C022
C023
C025
C026
C027
C028
C029
C030
C031


In [15]:
filename = 'dic_baseline_file_S2'
file=open(filename,'wb')
pickle.dump(dic_features_b_S2,file)
file.close()

In [16]:
dic_features_f_S1 = all_preprocessed_features(dict_emotion_S1, 'Fear', sampling_rate) 

C001
C002
C003
C004
C005
C006
C007


C:\Users\tiago\anaconda3\lib\site-packages\numpy\core\fromnumeric.py:3440: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\tiago\anaconda3\lib\site-packages\numpy\core\_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


C008


C:\Users\tiago\anaconda3\lib\site-packages\numpy\core\fromnumeric.py:3440: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\tiago\anaconda3\lib\site-packages\numpy\core\_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


C009
C010
C011
C012
C013
C014
C015
C016
C018
C019
C020
C021
C022
C023
C025
C026
C027
C028
C029
C030
C031


C:\Users\tiago\anaconda3\lib\site-packages\numpy\core\fromnumeric.py:3440: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\tiago\anaconda3\lib\site-packages\numpy\core\_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [17]:
filename = 'dic_fear_file_S1'
file=open(filename,'wb')
pickle.dump(dic_features_f_S1,file)
file.close()

In [18]:
dic_features_f_S2 = all_preprocessed_features(dict_emotion_S2, 'Fear', sampling_rate) 

C001
C002
C003
C004
C005
C006
C007
C008
C009
C010
C011
C012
C013
C014
C015
C016
C018
C019
C020
C021
C022
C023
C025
C026


C:\Users\tiago\anaconda3\lib\site-packages\numpy\core\fromnumeric.py:3440: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\tiago\anaconda3\lib\site-packages\numpy\core\_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


C027
C028
C029
C030
C031


In [19]:
filename = 'dic_fear_file_S2'
file=open(filename,'wb')
pickle.dump(dic_features_f_S2,file)
file.close()

In [20]:
dic_features_h_S1 = all_preprocessed_features(dict_emotion_S1, 'Happy', sampling_rate) 

C001
C002


C:\Users\tiago\anaconda3\lib\site-packages\numpy\core\fromnumeric.py:3440: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\tiago\anaconda3\lib\site-packages\numpy\core\_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


C003
C004
C005
C006
C007
C008
C009
C010
C011
C012
C013
C014
C015
C016


C:\Users\tiago\anaconda3\lib\site-packages\numpy\core\fromnumeric.py:3440: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\tiago\anaconda3\lib\site-packages\numpy\core\_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


C018
C019
C020
C021
C022
C023
C025
C026
C027
C028
C029
C030
C031


In [21]:
filename = 'dic_happy_file_S1'
file=open(filename,'wb')
pickle.dump(dic_features_h_S1,file)
file.close()

In [22]:
dic_features_h_S2 = all_preprocessed_features(dict_emotion_S2, 'Happy', sampling_rate) 

C001
C002
C003
C004
C005
C006
C007
C008
C009
C010
C011
C012


C:\Users\tiago\anaconda3\lib\site-packages\numpy\core\fromnumeric.py:3440: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\tiago\anaconda3\lib\site-packages\numpy\core\_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


C013
C014
C015
C016
C018
C019
C020


C:\Users\tiago\anaconda3\lib\site-packages\numpy\core\fromnumeric.py:3440: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\tiago\anaconda3\lib\site-packages\numpy\core\_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


C021
C022
C023
C025
C026
C027


C:\Users\tiago\anaconda3\lib\site-packages\numpy\core\fromnumeric.py:3440: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\tiago\anaconda3\lib\site-packages\numpy\core\_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


C028
C029
C030
C031


In [23]:
filename = 'dic_happy_file_S2'
file=open(filename,'wb')
pickle.dump(dic_features_h_S2,file)
file.close()

In [24]:
dic_features_n_S1 = all_preprocessed_features (dict_emotion_S1, 'Neutral', sampling_rate)

C001
C002
C003
C004
C005
C006
C007
C008
C009
C010
C011


C:\Users\tiago\anaconda3\lib\site-packages\numpy\core\fromnumeric.py:3440: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\tiago\anaconda3\lib\site-packages\numpy\core\_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


C012
C013
C014
C015
C016
C018
C019
C020
C021
C022
C023
C025
C026
C027
C028
C029
C030
C031


In [25]:
filename = 'dic_neutral_file_S1'
file=open(filename,'wb')
pickle.dump(dic_features_n_S1,file)
file.close()

In [26]:
dic_features_n_S2 = all_preprocessed_features (dict_emotion_S2, 'Neutral', sampling_rate)

C001
C002


C:\Users\tiago\anaconda3\lib\site-packages\numpy\core\fromnumeric.py:3440: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\tiago\anaconda3\lib\site-packages\numpy\core\_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


C003
C004
C005
C006
C007
C008
C009
C010
C011
C012
C013
C014
C015
C016
C018
C019
C020
C021
C022
C023
C025
C026
C027
C028
C029
C030
C031


In [27]:
filename = 'dic_neutral_file_S2'
file=open(filename,'wb')
pickle.dump(dic_features_n_S2,file)
file.close()

In [28]:
dic_baseline_S1 = pickle.load(open('dic_baseline_file_S1', 'rb'))
dic_fear_S1 = pickle.load(open('dic_fear_file_S1', 'rb'))
dic_happy_S1 = pickle.load(open('dic_happy_file_S1', 'rb'))
dic_neutral_S1 = pickle.load(open('dic_neutral_file_S1', 'rb'))

dic_baseline_S2 = pickle.load(open('dic_baseline_file_S2', 'rb'))
dic_fear_S2 = pickle.load(open('dic_fear_file_S2', 'rb'))
dic_happy_S2 = pickle.load(open('dic_happy_file_S2', 'rb'))
dic_neutral_S2 = pickle.load(open('dic_neutral_file_S2', 'rb'))

In [29]:
def all_features (dic_features, sampling_rate):
    
    lista=[]
        
    for participant in dic_features.keys():  
        
        # Features
        features_emg_mf = feature_extraction (dic_features[participant], 'EMG_MF', sampling_rate)
        features_emg_tr = feature_extraction (dic_features[participant], 'EMG_TR', sampling_rate)
        features_eda = feature_extraction (dic_features[participant], 'EDA', sampling_rate)
        features_ecg = feature_extraction (dic_features[participant], 'ECG', sampling_rate)
    
        part=pd.DataFrame({'ID participant':[participant]})
        features_=pd.concat([features_emg_mf, features_emg_tr, features_eda, features_ecg],axis=1)
        lista.append(features_)    
      
    return lista

In [31]:
b_features_S1 = all_features(dic_features_b_S1, sampling_rate)
b_df_S1=pd.DataFrame()
baseline_features_S1 = b_df_S1.append(other=b_features_S1,ignore_index=True)

C:\Users\tiago\AppData\Local\Temp\ipykernel_18384\1073071401.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  baseline_features_S1 = b_df_S1.append(other=b_features_S1,ignore_index=True)


In [32]:
b_features_S2 = all_features(dic_features_b_S2, sampling_rate)
b_df_S2=pd.DataFrame()
baseline_features_S2 = b_df_S2.append(other=b_features_S2,ignore_index=True)

C:\Users\tiago\AppData\Local\Temp\ipykernel_18384\1331236163.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  baseline_features_S2 = b_df_S2.append(other=b_features_S2,ignore_index=True)


In [33]:
f_features_S1 = all_features(dic_features_f_S1, sampling_rate)
f_df_S1=pd.DataFrame()
fear_features_S1 = f_df_S1.append(other=f_features_S1,ignore_index=True)

C:\Users\tiago\AppData\Local\Temp\ipykernel_18384\3596490305.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  fear_features_S1 = f_df_S1.append(other=f_features_S1,ignore_index=True)


In [34]:
f_features_S2 = all_features(dic_features_f_S2, sampling_rate)
f_df_S2=pd.DataFrame()
fear_features_S2 = f_df_S2.append(other=f_features_S2,ignore_index=True)

C:\Users\tiago\OneDrive - Universidade de Aveiro\3- Data Collection1\neurokit_function.py:206: RuntimeWarning: invalid value encountered in double_scalars
  skewness = (3*(mean-np.nanmedian(a)))/standard_deviation
C:\Users\tiago\AppData\Local\Temp\ipykernel_18384\564299775.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  fear_features_S2 = f_df_S2.append(other=f_features_S2,ignore_index=True)


In [35]:
h_features_S1 = all_features(dic_features_h_S1, sampling_rate)
h_df_S1=pd.DataFrame()
happy_features_S1 = h_df_S1.append(other=h_features_S1,ignore_index=True)

C:\Users\tiago\AppData\Local\Temp\ipykernel_18384\964619616.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  happy_features_S1 = h_df_S1.append(other=h_features_S1,ignore_index=True)


In [36]:
h_features_S2 = all_features(dic_features_h_S2, sampling_rate)
h_df_S2=pd.DataFrame()
happy_features_S2 = h_df_S2.append(other=h_features_S2,ignore_index=True)

C:\Users\tiago\AppData\Local\Temp\ipykernel_18384\3083990542.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  happy_features_S2 = h_df_S2.append(other=h_features_S2,ignore_index=True)


In [37]:
n_features_S1 = all_features(dic_features_n_S1, sampling_rate)
n_df_S1=pd.DataFrame()
neutral_features_S1 = n_df_S1.append(other=n_features_S1,ignore_index=True)

C:\Users\tiago\AppData\Local\Temp\ipykernel_18384\3607637079.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  neutral_features_S1 = n_df_S1.append(other=n_features_S1,ignore_index=True)


In [38]:
n_features_S2 = all_features(dic_features_n_S2, sampling_rate)
n_df_S2=pd.DataFrame()
neutral_features_S2 = n_df_S2.append(other=n_features_S2,ignore_index=True)

C:\Users\tiago\OneDrive - Universidade de Aveiro\3- Data Collection1\neurokit_function.py:206: RuntimeWarning: invalid value encountered in double_scalars
  skewness = (3*(mean-np.nanmedian(a)))/standard_deviation
C:\Users\tiago\OneDrive - Universidade de Aveiro\3- Data Collection1\neurokit_function.py:206: RuntimeWarning: invalid value encountered in double_scalars
  skewness = (3*(mean-np.nanmedian(a)))/standard_deviation
C:\Users\tiago\AppData\Local\Temp\ipykernel_18384\4037562465.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  neutral_features_S2 = n_df_S2.append(other=n_features_S2,ignore_index=True)


In [39]:
file_feat_S1='conditions_features_S1.xlsx'
writer=pd.ExcelWriter(file_feat_S1, engine='xlsxwriter')
baseline_features_S1.to_excel(writer, sheet_name='Baseline_S1', na_rep='nan')
fear_features_S1.to_excel(writer, sheet_name='Fear_S1', na_rep='nan')
happy_features_S1.to_excel(writer, sheet_name='Happy_S1', na_rep='nan')
neutral_features_S1.to_excel(writer, sheet_name='Neutral_S1', na_rep='nan')
writer.save()

In [40]:
file_feat_S2='conditions_features_S2.xlsx'
writer=pd.ExcelWriter(file_feat_S2, engine='xlsxwriter')
baseline_features_S2.to_excel(writer, sheet_name='Baseline_S2', na_rep='nan')
fear_features_S2.to_excel(writer, sheet_name='Fear_S2', na_rep='nan')
happy_features_S2.to_excel(writer, sheet_name='Happy_S2', na_rep='nan')
neutral_features_S2.to_excel(writer, sheet_name='Neutral_S2', na_rep='nan')
writer.save()

In [3]:
file_feat='conditions_features_S1.xlsx'


In [4]:
baseline_S1 = pd.read_excel(file_feat, sheet_name='Baseline_S1',  usecols=lambda x: 'Unnamed' not in x)
fear_S1 = pd.read_excel(file_feat, sheet_name='Fear_S1',  usecols=lambda x: 'Unnamed' not in x)
happy_S1 = pd.read_excel(file_feat, sheet_name='Happy_S1',  usecols=lambda x: 'Unnamed' not in x)
neutral_S1 = pd.read_excel(file_feat, sheet_name='Neutral_S1',  usecols=lambda x: 'Unnamed' not in x)

file_feat='conditions_features_S2.xlsx'
baseline_S2 = pd.read_excel(file_feat, sheet_name='Baseline_S2',  usecols=lambda x: 'Unnamed' not in x)
fear_S2 = pd.read_excel(file_feat, sheet_name='Fear_S2',  usecols=lambda x: 'Unnamed' not in x)
happy_S2 = pd.read_excel(file_feat, sheet_name='Happy_S2',  usecols=lambda x: 'Unnamed' not in x)
neutral_S2 = pd.read_excel(file_feat, sheet_name='Neutral_S2',  usecols=lambda x: 'Unnamed' not in x)

In [5]:
print(baseline_S1)

    EMG_MF_Activations_N  EMG_MF_Duration_Mean  EMG_MF_Duration_Std  \
0                    155              0.071923             0.023539   
1                    216              0.086880             0.059227   
2                    246              0.106650             0.232876   
3                    213              0.082272             0.072255   
4                    156              0.110551             0.132471   
5                    106              0.110349             0.184508   
6                    269              0.090093             0.063360   
7                     84              0.272595             0.502232   
8                     94              0.088840             0.081815   
9                    153              0.151405             0.135776   
10                   265              0.112558             0.095671   
11                   137              0.122000             0.142367   
12                    77              0.154857             0.228275   
13    

In [6]:
print(baseline_S2)

    EMG_MF_Activations_N  EMG_MF_Duration_Mean  EMG_MF_Duration_Std  \
0                    130              0.109985             0.092575   
1                     93              0.080688             0.082498   
2                    120              0.074508             0.023734   
3                    170              0.100429             0.140026   
4                    295              0.086308             0.055902   
5                     87              0.165529             0.247944   
6                    295              0.088756             0.059491   
7                    161              0.166391             0.350268   
8                    159              0.125365             0.298972   
9                     96              0.176167             0.434926   
10                   166              0.084181             0.050544   
11                   147              0.121286             0.205839   
12                    81              0.160926             0.228561   
13    